<a href="https://colab.research.google.com/github/wlrma0108/Pytorch/blob/main/Convolution_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch

In [27]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [28]:
device=torch.device('cuda:0'if torch.cuda.is_available() else 'cpu')


In [29]:
num_epochs=5
num_classes =10
batch_size =100
learning_rate=0.001

In [38]:
train_dataset=torchvision.datasets.MNIST(root='../../data',train=True, transform=transforms.ToTensor(), download=True)
test_dataset=torchvision.datasets.MNIST(root='../../data', train=False, transform=transforms.ToTensor())
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [60]:
from torch.nn.modules.batchnorm import BatchNorm2d
class ConvNet(nn.Module):
  def __init__(self,num_classes=10):
    super(ConvNet,self).__init__()
    self.layer1= nn.Sequential(
        nn.Conv2d(1,16,kernel_size=5,stride=1,padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2))
    self.layer2=nn.Sequential(
        nn.Conv2d(16,32,kernel_size=5,stride=1, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
    self.fc=nn.Linear(7*7*32,num_classes)    
    


  def forward(self,x):
      out= self.layer1(x)
      out= self.layer2(out)
      out=out.reshape(out.size(0),-1)
      out=self.fc(out)
      return out


    

In [61]:
model = ConvNet(num_classes).to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

In [62]:
total_step=len(train_loader)
for epoch in range(num_epochs):
  for i,(images,labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    
    outputs = model(images)
    loss=criterion(outputs,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) %100 ==0:
      print('Epoch [{}/{},step [{}/{}],Loss:{:.4f}'.format(epoch+1,num_epochs,i+1,total_step,loss.item()))


  model.eval()
  with torch.no_grad():
    correct=0
    total=0
    for images,labels in test_loader:
      images=images.to(device)
      labels =labels.to(device)
      outputs = model(images)
      _,predicted = torch.max(outputs.data,1)
      total+=labels.size(0)
      correct+=(predicted==labels).sum().item()


    print('Test Accuracy of the model in the 10000 test images: {} %'.format(100*correct/total))


Epoch [1/5,step [100/600],Loss:0.1154
Epoch [1/5,step [200/600],Loss:0.0769
Epoch [1/5,step [300/600],Loss:0.1621
Epoch [1/5,step [400/600],Loss:0.2081
Epoch [1/5,step [500/600],Loss:0.0380
Epoch [1/5,step [600/600],Loss:0.0700
Test Accuracy of the model in the 10000 test images: 98.1 %
Epoch [2/5,step [100/600],Loss:0.0939
Epoch [2/5,step [200/600],Loss:0.0243
Epoch [2/5,step [300/600],Loss:0.0089
Epoch [2/5,step [400/600],Loss:0.0527
Epoch [2/5,step [500/600],Loss:0.0400
Epoch [2/5,step [600/600],Loss:0.0124
Test Accuracy of the model in the 10000 test images: 98.77 %
Epoch [3/5,step [100/600],Loss:0.0113
Epoch [3/5,step [200/600],Loss:0.0144
Epoch [3/5,step [300/600],Loss:0.0197
Epoch [3/5,step [400/600],Loss:0.0056
Epoch [3/5,step [500/600],Loss:0.0083
Epoch [3/5,step [600/600],Loss:0.0042
Test Accuracy of the model in the 10000 test images: 98.8 %
Epoch [4/5,step [100/600],Loss:0.0439
Epoch [4/5,step [200/600],Loss:0.0108
Epoch [4/5,step [300/600],Loss:0.0392
Epoch [4/5,step [400/

In [ ]:
torch.save(model.state_dict(), 'model.ckpt')